In [37]:
from pathlib import Path

import numpy as np
import preprocess_mesh as pm
import pyvista as pv

# Load Dataset

In [38]:
patient_id = "patient_01"
infile_name = Path(f"../data/raw/{patient_id}_mesh_with_uacs_fibers_tags.vtk")
outfile_name = Path(f"../data/processed/{patient_id}/mesh_with_fibers_tags.vtk")
legacy_anatomical_tags = {
    "Body": 11,
    "LAA": 13,
    "LIPV": 21,
    "LSPV": 23,
    "RIPV": 25,
    "RSPV": 27,
}

new_anatomical_tags = {
    "Body": 0,
    "LAA": 1,
    "LIPV": 2,
    "LSPV": 3,
    "RIPV": 4,
    "RSPV": 5,
}

input_mesh = pv.read(infile_name)
triangular_mesh = pm.convert_unstructured_to_polydata_mesh(input_mesh)
del triangular_mesh.point_data["alpha"]
del triangular_mesh.point_data["beta"]
for key, value in legacy_anatomical_tags.items():
    anatomical_region = np.where(triangular_mesh.cell_data["anatomical_tags"] == value)[0]
    triangular_mesh.cell_data["anatomical_tags"][anatomical_region] = new_anatomical_tags[key]

In [ ]:
coarse_mesh = pm.coarsen_mesh(triangular_mesh, decimation_factor=0.8)
coarse_mesh = pm.fix_feature_tags_after_interpolation(coarse_mesh, extraction_threshold=0.5)
coarse_mesh = pm.smoothen_feature_boundaries(coarse_mesh)
coarse_mesh = pm.fix_feature_tags_after_interpolation(coarse_mesh, extraction_threshold=0.5)
coarse_mesh = pm.remove_boundary_spikes(coarse_mesh)
coarse_mesh = pm.remove_feature_boundary_spikes(coarse_mesh)
coarse_mesh

PolyData (0x7f59bc1b9120)
  N Cells:    12465
  N Points:   6479
  N Strips:   0
  X Bounds:   2.787e+04, 6.993e+04
  Y Bounds:   1.670e+04, 6.522e+04
  Z Bounds:   1.284e+04, 5.690e+04
  N Arrays:   4

In [40]:
outfile_name.parent.mkdir(parents=True, exist_ok=True)
pv.save_meshio(outfile_name, coarse_mesh)

In [41]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(
    coarse_mesh,
    show_edges=True,
    scalars="anatomical_tags",
    cmap="tab20",
    show_scalar_bar=False,
    edge_color="lightgray",
    edge_opacity=0.3,
)
plotter.show()

Widget(value='<iframe src="http://localhost:37477/index.html?ui=P_0x7f5a4813a990_12&reconnect=auto" class="pyv…